# Notas de la tutoría

https://docs.google.com/document/d/1OFimhBpQRmyhBzsIPMzMCz5L3U5bnUaSXxvbL6H2dS0/edit?usp=sharing

# Importar librerías

In [ ]:
from google.colab import drive
drivedir = r'/content/drive'
drive.mount(drivedir)

import os
import numpy as np
import pandas as pd
import datetime as dt


# Configuración
pd.pandas.set_option('display.max_columns', None)
rootdir = os.path.join(drivedir, '/content/drive/MyDrive/Colab Notebooks/Store Sales')

Mounted at /content/drive


# Cargar datos

Vamos a eliminar ruido quitando la primera mitad de las series temporales, ya que tiene una tendencia diferente a la segunda mitad, pudiendo generar ruido.

También establecemos los tipos de datos correctos

In [ ]:
data_processed = pd.read_csv(os.path.join(rootdir, 'data_processed_2.csv'))
test_processed = pd.read_csv(os.path.join(rootdir, 'test_processed_2.csv'))
data_processed['date'] = pd.to_datetime(data_processed['date'])
test_processed['date'] = pd.to_datetime(test_processed['date'])

# Usamos solo los dos últimos años de datos
data_processed = data_processed[~(data_processed.date < "2015-08-16")]


holidays_events=pd.read_csv(os.path.join(rootdir, 'holidays_events.csv'))
holidays_events['date'] = pd.to_datetime(holidays_events['date'])
holidays_events.set_index('date', inplace=True)
# Quitar las vacaciones "transferred"
holidays_events = holidays_events.loc[holidays_events.iloc[:, -1] != "True"]

oil_processed=pd.read_csv(os.path.join(rootdir, 'oil_processed.csv'))
oil_processed['date'] = pd.to_datetime(oil_processed['date'])
oil_processed.set_index('date', inplace=True)

In [ ]:
display(data_processed)
display(test_processed)

,date,store_nbr,family,sales,onpromotion,city,state,type,cluster,dcoilwtico,event_earthquake
1701810,2015-08-16,1,AUTOMOTIVE,1.000,0,Quito,Pichincha,D,13,42.103333,0
1701811,2015-08-16,1,BABY CARE,0.000,0,Quito,Pichincha,D,13,42.103333,0
1701812,2015-08-16,1,BEAUTY,2.000,0,Quito,Pichincha,D,13,42.103333,0
1701813,2015-08-16,1,BEVERAGES,813.000,3,Quito,Pichincha,D,13,42.103333,0
1701814,2015-08-16,1,BOOKS,0.000,0,Quito,Pichincha,D,13,42.103333,0
...,...,...,...,...,...,...,...,...,...,...,...
3000883,2017-08-15,9,POULTRY,438.133,0,Quito,Pichincha,B,6,47.570000,0
3000884,2017-08-15,9,PREPARED FOODS,154.553,1,Quito,Pichincha,B,6,47.570000,0
3000885,2017-08-15,9,PRODUCE,2419.729,148,Quito,Pichincha,B,6,47.570000,0
3000886,2017-08-15,9,SCHOOL AND OFFICE SUPPLIES,121.000,8,Quito,Pichincha,B,6,47.570000,0


,date,store_nbr,family,onpromotion,city,state,type,cluster,dcoilwtico,event_earthquake
0,2017-08-16,1,AUTOMOTIVE,0,Quito,Pichincha,D,13,46.80,0
1,2017-08-16,1,BABY CARE,0,Quito,Pichincha,D,13,46.80,0
2,2017-08-16,1,BEAUTY,2,Quito,Pichincha,D,13,46.80,0
3,2017-08-16,1,BEVERAGES,20,Quito,Pichincha,D,13,46.80,0
4,2017-08-16,1,BOOKS,0,Quito,Pichincha,D,13,46.80,0
...,...,...,...,...,...,...,...,...,...,...
28507,2017-08-31,9,POULTRY,1,Quito,Pichincha,B,6,47.26,0
28508,2017-08-31,9,PREPARED FOODS,0,Quito,Pichincha,B,6,47.26,0
28509,2017-08-31,9,PRODUCE,1,Quito,Pichincha,B,6,47.26,0
28510,2017-08-31,9,SCHOOL AND OFFICE SUPPLIES,9,Quito,Pichincha,B,6,47.26,0


# Procesar datos

Combinar las vacaciones

Añadimos nueva feature con el día de la semana

Eliminamos columnas redundantes o que aportan ruido

In [ ]:
data_processed = data_processed.merge(holidays_events, on='date', how='left')
test_processed = test_processed.merge(holidays_events, on='date', how='left')

def nat_holiday(row):
    if row['locale'] == 'National':
        return 1
    else:
        return 0
def reg_holiday(row):
    if row['locale'] == 'Regional' and row['locale_name'] == row['state']:
        return 1
    else:
        return 0
def loc_holiday(row):
    if row['locale'] == 'Local' and row['locale_name'] == row['city']:
        return 1
    else:
        return 0

data_processed['holiday_national'] = data_processed.apply(nat_holiday, axis=1)
data_processed['holiday_regional'] = data_processed.apply(reg_holiday, axis=1)
data_processed['holiday_local'] = data_processed.apply(loc_holiday, axis=1)

test_processed['holiday_national'] = test_processed.apply(nat_holiday, axis=1)
test_processed['holiday_regional'] = test_processed.apply(reg_holiday, axis=1)
test_processed['holiday_local'] = test_processed.apply(loc_holiday, axis=1)

In [ ]:
data_processed['day_of_week'] = data_processed['date'].dt.day_name()
test_processed['day_of_week'] = test_processed['date'].dt.day_name()

In [ ]:
data_processed = data_processed.drop(columns=['city', 'state', 'type_y', 'locale', 'locale_name', 'description', 'transferred'])
test_processed = test_processed.drop(columns=['city', 'state', 'type_y', 'locale', 'locale_name', 'description', 'transferred'])

In [ ]:
display(data_processed)
display(test_processed)

,date,store_nbr,family,sales,onpromotion,type_x,cluster,dcoilwtico,event_earthquake,holiday_national,holiday_regional,holiday_local,day_of_week
0,2015-08-16,1,AUTOMOTIVE,1.000,0,D,13,42.103333,0,0,0,0,Sunday
1,2015-08-16,1,BABY CARE,0.000,0,D,13,42.103333,0,0,0,0,Sunday
2,2015-08-16,1,BEAUTY,2.000,0,D,13,42.103333,0,0,0,0,Sunday
3,2015-08-16,1,BEVERAGES,813.000,3,D,13,42.103333,0,0,0,0,Sunday
4,2015-08-16,1,BOOKS,0.000,0,D,13,42.103333,0,0,0,0,Sunday
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1327585,2017-08-15,9,POULTRY,438.133,0,B,6,47.570000,0,0,0,0,Tuesday
1327586,2017-08-15,9,PREPARED FOODS,154.553,1,B,6,47.570000,0,0,0,0,Tuesday
1327587,2017-08-15,9,PRODUCE,2419.729,148,B,6,47.570000,0,0,0,0,Tuesday
1327588,2017-08-15,9,SCHOOL AND OFFICE SUPPLIES,121.000,8,B,6,47.570000,0,0,0,0,Tuesday


,date,store_nbr,family,onpromotion,type_x,cluster,dcoilwtico,event_earthquake,holiday_national,holiday_regional,holiday_local,day_of_week
0,2017-08-16,1,AUTOMOTIVE,0,D,13,46.80,0,0,0,0,Wednesday
1,2017-08-16,1,BABY CARE,0,D,13,46.80,0,0,0,0,Wednesday
2,2017-08-16,1,BEAUTY,2,D,13,46.80,0,0,0,0,Wednesday
3,2017-08-16,1,BEVERAGES,20,D,13,46.80,0,0,0,0,Wednesday
4,2017-08-16,1,BOOKS,0,D,13,46.80,0,0,0,0,Wednesday
...,...,...,...,...,...,...,...,...,...,...,...,...
28507,2017-08-31,9,POULTRY,1,B,6,47.26,0,0,0,0,Thursday
28508,2017-08-31,9,PREPARED FOODS,0,B,6,47.26,0,0,0,0,Thursday
28509,2017-08-31,9,PRODUCE,1,B,6,47.26,0,0,0,0,Thursday
28510,2017-08-31,9,SCHOOL AND OFFICE SUPPLIES,9,B,6,47.26,0,0,0,0,Thursday


# Entrenamiento con Prophet

Se trata de un algoritmo especializado en series temporales que controla bien la estacionalidad

In [ ]:
!pip install prophet
import prophet

In [ ]:
# Preparamos los nombres de columnas para Prophet
data_processed.rename(columns={'date':'ds', 'sales':'y'}, inplace=True)
test_processed.rename(columns={'date':'ds'}, inplace=True)

In [ ]:
display(data_processed)
display(test_processed)
display(holidays_events)
display(oil_processed)

,ds,store_nbr,family,y,onpromotion,type_x,cluster,dcoilwtico,event_earthquake,holiday_national,holiday_regional,holiday_local,day_of_week
0,2015-08-16,1,AUTOMOTIVE,1.000,0,D,13,42.103333,0,0,0,0,Sunday
1,2015-08-16,1,BABY CARE,0.000,0,D,13,42.103333,0,0,0,0,Sunday
2,2015-08-16,1,BEAUTY,2.000,0,D,13,42.103333,0,0,0,0,Sunday
3,2015-08-16,1,BEVERAGES,813.000,3,D,13,42.103333,0,0,0,0,Sunday
4,2015-08-16,1,BOOKS,0.000,0,D,13,42.103333,0,0,0,0,Sunday
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1327585,2017-08-15,9,POULTRY,438.133,0,B,6,47.570000,0,0,0,0,Tuesday
1327586,2017-08-15,9,PREPARED FOODS,154.553,1,B,6,47.570000,0,0,0,0,Tuesday
1327587,2017-08-15,9,PRODUCE,2419.729,148,B,6,47.570000,0,0,0,0,Tuesday
1327588,2017-08-15,9,SCHOOL AND OFFICE SUPPLIES,121.000,8,B,6,47.570000,0,0,0,0,Tuesday


,ds,store_nbr,family,onpromotion,type_x,cluster,dcoilwtico,event_earthquake,holiday_national,holiday_regional,holiday_local,day_of_week
0,2017-08-16,1,AUTOMOTIVE,0,D,13,46.80,0,0,0,0,Wednesday
1,2017-08-16,1,BABY CARE,0,D,13,46.80,0,0,0,0,Wednesday
2,2017-08-16,1,BEAUTY,2,D,13,46.80,0,0,0,0,Wednesday
3,2017-08-16,1,BEVERAGES,20,D,13,46.80,0,0,0,0,Wednesday
4,2017-08-16,1,BOOKS,0,D,13,46.80,0,0,0,0,Wednesday
...,...,...,...,...,...,...,...,...,...,...,...,...
28507,2017-08-31,9,POULTRY,1,B,6,47.26,0,0,0,0,Thursday
28508,2017-08-31,9,PREPARED FOODS,0,B,6,47.26,0,0,0,0,Thursday
28509,2017-08-31,9,PRODUCE,1,B,6,47.26,0,0,0,0,Thursday
28510,2017-08-31,9,SCHOOL AND OFFICE SUPPLIES,9,B,6,47.26,0,0,0,0,Thursday


,type,locale,locale_name,description,transferred
date,,,,,
2012-03-02,Holiday,Local,Manta,Fundacion de Manta,False
2012-04-01,Holiday,Regional,Cotopaxi,Provincializacion de Cotopaxi,False
2012-04-12,Holiday,Local,Cuenca,Fundacion de Cuenca,False
2012-04-14,Holiday,Local,Libertad,Cantonizacion de Libertad,False
2012-04-21,Holiday,Local,Riobamba,Cantonizacion de Riobamba,False
...,...,...,...,...,...
2017-12-22,Additional,National,Ecuador,Navidad-3,False
2017-12-23,Additional,National,Ecuador,Navidad-2,False
2017-12-24,Additional,National,Ecuador,Navidad-1,False


,dcoilwtico
date,
2013-01-01,93.140000
2013-01-02,93.140000
2013-01-03,92.970000
2013-01-04,93.120000
2013-01-05,93.146667
...,...
2017-08-27,46.816667
2017-08-28,46.400000
2017-08-29,46.460000


In [ ]:
from prophet import Prophet
from prophet.diagnostics import cross_validation, performance_metrics
from prophet.plot import plot_plotly, plot_components_plotly, add_changepoints_to_plot
import itertools
from pandas import Timestamp

33
54


## Prueba con una muestra reducida

In [ ]:
# Prueba con una muestra reducida
# tiendas = [1,22,33]
# familias = ['AUTOMOTIVE', 'BEAUTY', 'BEVERAGES']

## Entrenamiento con el dataset completo

In [ ]:
familias = data_processed.family.unique()
tiendas = data_processed.store_nbr.unique()

print(np.count_nonzero(familias))
print(np.count_nonzero(tiendas))

In [ ]:
full_forecast = pd.DataFrame()
empty_series_log = pd.DataFrame() # Para debugar

for t in tiendas:
  print('TIENDA: ' + str(t) + '-------------------------------------')

  for f in familias:
    print('FAMILIA: ' + str(f) + '-------------------------------------')

    serie = data_processed.loc[(data_processed['family'] == f) & (data_processed['store_nbr'] == t) ]
    if serie['ds'].count() > 0:
      m = Prophet(changepoint_prior_scale=10)
      m.fit(serie)
      future = m.make_future_dataframe(periods=16)
      forecast = m.predict(future)
      forecast_selected_columns = forecast[['ds', 'yhat']].tail(16)
      forecast_selected_columns['store_nbr'] = t
      forecast_selected_columns['family'] = f
      full_forecast = pd.concat([full_forecast, forecast_selected_columns])
      # plot_plotly(m, forecast).show()
      # fig = m.plot(forecast)
      # a = add_changepoints_to_plot(fig.gca(), m, forecast)

    else:
      new_log = {'store_nbr':t, 'family':f}
      empty_series_log = empty_series_log.append(new_log, ignore_index=True)



# Guardamos el fichero para su tratamiento posterior
# Ahora está desordenado, habrá que ordenarlo y darle el id correcto a cada fila
full_forecast.to_csv( os.path.join( rootdir, 'forecast_temp_3.csv'), index=False, encoding='utf-8' )
empty_series_log.to_csv( os.path.join( rootdir, 'empty_series_log_3.csv'), index=False, encoding='utf-8' )

TIENDA: 1-------------------------------------
FAMILIA: AUTOMOTIVE-------------------------------------


INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
DEBUG:cmdstanpy:input tempfile: /tmp/tmpsovwnai5/6hgzbjj4.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpsovwnai5/my2drsos.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.10/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=69959', 'data', 'file=/tmp/tmpsovwnai5/6hgzbjj4.json', 'init=/tmp/tmpsovwnai5/my2drsos.json', 'output', 'file=/tmp/tmpsovwnai5/prophet_modelscn9vjci/prophet_model-20240102125638.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
12:56:38 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
12:56:38 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


FAMILIA: BABY CARE-------------------------------------


INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
DEBUG:cmdstanpy:input tempfile: /tmp/tmpsovwnai5/tg0mr4cp.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpsovwnai5/5ca24ojh.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.10/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=78441', 'data', 'file=/tmp/tmpsovwnai5/tg0mr4cp.json', 'init=/tmp/tmpsovwnai5/5ca24ojh.json', 'output', 'file=/tmp/tmpsovwnai5/prophet_modeljqu93x_2/prophet_model-20240102125639.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
12:56:39 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing


FAMILIA: BEAUTY-------------------------------------


...

INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
DEBUG:cmdstanpy:input tempfile: /tmp/tmpsovwnai5/7vadyw1e.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpsovwnai5/iuirf3al.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.10/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=73042', 'data', 'file=/tmp/tmpsovwnai5/7vadyw1e.json', 'init=/tmp/tmpsovwnai5/iuirf3al.json', 'output', 'file=/tmp/tmpsovwnai5/prophet_modellgiuc5op/prophet_model-20240102132549.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
13:25:49 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
13:25:49 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


FAMILIA: SCHOOL AND OFFICE SUPPLIES-------------------------------------


DEBUG:cmdstanpy:input tempfile: /tmp/tmpsovwnai5/vmalvi17.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpsovwnai5/zjmpr701.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.10/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=33785', 'data', 'file=/tmp/tmpsovwnai5/vmalvi17.json', 'init=/tmp/tmpsovwnai5/zjmpr701.json', 'output', 'file=/tmp/tmpsovwnai5/prophet_modelcqaykwqy/prophet_model-20240102132550.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
13:25:50 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
13:25:51 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
DEBUG:cmdstanpy:input tempfile: /tmp/tmpsovwnai5/1zvhy4nc.json


FAMILIA: SEAFOOD-------------------------------------


DEBUG:cmdstanpy:input tempfile: /tmp/tmpsovwnai5/t3mbguck.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.10/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=50835', 'data', 'file=/tmp/tmpsovwnai5/1zvhy4nc.json', 'init=/tmp/tmpsovwnai5/t3mbguck.json', 'output', 'file=/tmp/tmpsovwnai5/prophet_modelz5zxvipp/prophet_model-20240102132551.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
13:25:51 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
13:25:52 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing


# Ordenar fichero para kaggle

In [ ]:
# Cargamos el fichero de predicciones para ordenarlo y combinarlo con el fichero de predicción cero
full_forecast = pd.read_csv(os.path.join(rootdir, 'forecast_temp_3.csv'))

In [ ]:
# Convierto store_nbr a string para que lo ordene alfabéticamente como en test.csv (1, 10, 11, 12 ... 54, 6, 7, 8, 9)
full_forecast['store_nbr'] = full_forecast['store_nbr'].astype(str)
full_forecast.sort_values(['ds', 'store_nbr', 'family'], ascending=[True, True, True], inplace=True)
full_forecast['yhat'][full_forecast['yhat'] < 0.01] = 0

submission = pd.DataFrame()
first_id = 3000888
submission.insert(0, 'id', range(first_id, first_id + len(full_forecast)))
full_forecast.index = submission.index
submission['sales'] = full_forecast['yhat']
submission.to_csv( os.path.join( rootdir, 'submission_3.csv'), index=False, encoding='utf-8' )

<ipython-input-63-b6ed18de94af>:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

